# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.
/Users/timsennett/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/timsennett/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/timsennett/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0        80.0         69.0  21453.0          6.0          5.0     1969.0   
1        60.0         79.0  12420.0          7.0          5.0     2001.0   
2        20.0         75.0   9742.0          8.0          5.0     2002.0   
3       120.0         39.0   5389.0          8.0          5.0     1995.0   
4        60.0         85.0  11003.0         10.0          5.0     2008.0   

   YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2          ...            \
0        1969.0         0.0       938.0         0.0          ...             
1        2001.0         0.0       666.0         0.0          ...             
2        2002.0       281.0         0.0         0.0          ...             
3        1996.0         0.0      1180.0         0.0          ...             
4        2008.0       160.0       765.0         0.0          ...             

   SaleType_ConLw  SaleType_New  SaleType_Oth  SaleType_WD  \
0             0.0           0.0           0.0          1.0   
1             0.0           0.0           0.0          1.0   
2             0.0           0.0           0.0          1.0   
3             0.0           0.0           0.0          1.0   
4             0.0           0.0           0.0          1.0   

   SaleCondition_Abnorml  SaleCondition_AdjLand  SaleCondition_Alloca  \
0                    0.0                    0.0                   0.0   
1                    0.0                    0.0                   0.0   
2                    0.0                    0.0                   0.0   
3                    0.0                    0.0                   0.0   
4                    0.0                    0.0                   0.0   

   SaleCondition_Family  SaleCondition_Normal  SaleCondition_Partial  
0                   0.0                   1.0                    0.0  
1                   0.0                   1.0                    0.0  
2                   0.0                   1.0                    0.0  
3                   0.0                   1.0                    0.0  
4                   0.0                   1.0                    0.0  

[5 rows x 296 columns]

In [5]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1051 entries, 0 to 1050
Columns: 296 entries, MSSubClass to SaleCondition_Partial
dtypes: float64(296)
memory usage: 2.4 MB


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [6]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 199us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 88us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 93us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 111us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 90us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 121us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [===

1051/1051 [==============================] - 0s 76us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 100us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 149us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 80us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 82us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 74us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 78us/step - loss: na

Epoch 106/150
1051/1051 [==============================] - 0s 72us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 72us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 70us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 71us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 72us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 73us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 112/150
1051/1051 [==============================] - 0s 71

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [7]:
# Numeric column names
numeric_columns = X_train_numeric.columns

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# # Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# # Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [8]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [9]:
# Train the model 
normalized_input_model.fit(X_train,
                           y_train,
                           batch_size=32,
                           epochs=150,
                           validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 237us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 81us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 109us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 102us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 97us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 80us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [===

1051/1051 [==============================] - 0s 75us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 75us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 73us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 92us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 88us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 147us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 192us/step - loss: na

Epoch 106/150
1051/1051 [==============================] - 0s 131us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 152us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 131us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 145us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 104us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 81us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 112/150
1051/1051 [==============================] - 

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [10]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [11]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train,
                     y_train_scaled,
                     batch_size=32,
                     epochs=150,
                     validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 258us/step - loss: 0.4367 - mean_squared_error: 0.4367 - val_loss: 0.1997 - val_mean_squared_error: 0.1997
Epoch 2/150
1051/1051 [==============================] - 0s 90us/step - loss: 0.2272 - mean_squared_error: 0.2272 - val_loss: 0.1616 - val_mean_squared_error: 0.1616
Epoch 3/150
1051/1051 [==============================] - 0s 98us/step - loss: 0.1842 - mean_squared_error: 0.1842 - val_loss: 0.1589 - val_mean_squared_error: 0.1589
Epoch 4/150
1051/1051 [==============================] - 0s 116us/step - loss: 0.1639 - mean_squared_error: 0.1639 - val_loss: 0.1378 - val_mean_squared_error: 0.1378
Epoch 5/150
1051/1051 [==============================] - 0s 104us/step - loss: 0.1480 - mean_squared_error: 0.1480 - val_loss: 0.1319 - val_mean_squared_error: 0.1319
Epoch 6/150
1051/1051 [==============================] - 0s 107us/step - loss: 0.1331 - mean_squared_error: 0.1331 - val

1051/1051 [==============================] - 0s 99us/step - loss: 0.0329 - mean_squared_error: 0.0329 - val_loss: 0.1036 - val_mean_squared_error: 0.1036
Epoch 50/150
1051/1051 [==============================] - 0s 98us/step - loss: 0.0318 - mean_squared_error: 0.0318 - val_loss: 0.1016 - val_mean_squared_error: 0.1016
Epoch 51/150
1051/1051 [==============================] - 0s 92us/step - loss: 0.0318 - mean_squared_error: 0.0318 - val_loss: 0.1030 - val_mean_squared_error: 0.1030
Epoch 52/150
1051/1051 [==============================] - 0s 95us/step - loss: 0.0309 - mean_squared_error: 0.0309 - val_loss: 0.1032 - val_mean_squared_error: 0.1032
Epoch 53/150
1051/1051 [==============================] - 0s 142us/step - loss: 0.0303 - mean_squared_error: 0.0303 - val_loss: 0.1034 - val_mean_squared_error: 0.1034
Epoch 54/150
1051/1051 [==============================] - 0s 86us/step - loss: 0.0292 - mean_squared_error: 0.0292 - val_loss: 0.1053 - val_mean_squared_error: 0.1053
Epoch 55/1

Epoch 98/150
1051/1051 [==============================] - 0s 85us/step - loss: 0.0145 - mean_squared_error: 0.0145 - val_loss: 0.1104 - val_mean_squared_error: 0.1104
Epoch 99/150
1051/1051 [==============================] - 0s 83us/step - loss: 0.0144 - mean_squared_error: 0.0144 - val_loss: 0.1118 - val_mean_squared_error: 0.1118
Epoch 100/150
1051/1051 [==============================] - 0s 77us/step - loss: 0.0142 - mean_squared_error: 0.0142 - val_loss: 0.1111 - val_mean_squared_error: 0.1111
Epoch 101/150
1051/1051 [==============================] - 0s 79us/step - loss: 0.0141 - mean_squared_error: 0.0141 - val_loss: 0.1125 - val_mean_squared_error: 0.1125
Epoch 102/150
1051/1051 [==============================] - 0s 78us/step - loss: 0.0138 - mean_squared_error: 0.0138 - val_loss: 0.1117 - val_mean_squared_error: 0.1117
Epoch 103/150
1051/1051 [==============================] - 0s 98us/step - loss: 0.0137 - mean_squared_error: 0.0137 - val_loss: 0.1118 - val_mean_squared_error: 0

1051/1051 [==============================] - 0s 145us/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.1173 - val_mean_squared_error: 0.1173
Epoch 147/150
1051/1051 [==============================] - 0s 113us/step - loss: 0.0084 - mean_squared_error: 0.0084 - val_loss: 0.1178 - val_mean_squared_error: 0.1178
Epoch 148/150
1051/1051 [==============================] - 0s 144us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1182 - val_mean_squared_error: 0.1182
Epoch 149/150
1051/1051 [==============================] - 0s 109us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1177 - val_mean_squared_error: 0.1177
Epoch 150/150
1051/1051 [==============================] - 0s 119us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1189 - val_mean_squared_error: 0.1189


Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [12]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 67us/step


[0.007793592263835453, 0.007793592263835453]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [13]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 92us/step


[0.11888821746006664, 0.11888821746006664]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [14]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

27095.732817231325

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [16]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu',
                          input_shape=(n_features,),
                          kernel_initializer='he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 245us/step - loss: 0.5083 - mean_squared_error: 0.5083 - val_loss: 0.1704 - val_mean_squared_error: 0.1704
Epoch 2/150
1051/1051 [==============================] - 0s 88us/step - loss: 0.2216 - mean_squared_error: 0.2216 - val_loss: 0.1433 - val_mean_squared_error: 0.1433
Epoch 3/150
1051/1051 [==============================] - 0s 80us/step - loss: 0.1793 - mean_squared_error: 0.1793 - val_loss: 0.1376 - val_mean_squared_error: 0.1376
Epoch 4/150
1051/1051 [==============================] - 0s 112us/step - loss: 0.1569 - mean_squared_error: 0.1569 - val_loss: 0.1249 - val_mean_squared_error: 0.1249
Epoch 5/150
1051/1051 [==============================] - 0s 95us/step - loss: 0.1404 - mean_squared_error: 0.1404 - val_loss: 0.1196 - val_mean_squared_error: 0.1196
Epoch 6/150
1051/1051 [==============================] - 0s 107us/step - loss: 0.1270 - mean_squared_error: 0.1270 - val_

1051/1051 [==============================] - 0s 98us/step - loss: 0.0314 - mean_squared_error: 0.0314 - val_loss: 0.1110 - val_mean_squared_error: 0.1110
Epoch 50/150
1051/1051 [==============================] - 0s 74us/step - loss: 0.0302 - mean_squared_error: 0.0302 - val_loss: 0.1092 - val_mean_squared_error: 0.1092
Epoch 51/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.0296 - mean_squared_error: 0.0296 - val_loss: 0.1093 - val_mean_squared_error: 0.1093
Epoch 52/150
1051/1051 [==============================] - 0s 74us/step - loss: 0.0292 - mean_squared_error: 0.0292 - val_loss: 0.1104 - val_mean_squared_error: 0.1104
Epoch 53/150
1051/1051 [==============================] - 0s 81us/step - loss: 0.0287 - mean_squared_error: 0.0287 - val_loss: 0.1107 - val_mean_squared_error: 0.1107
Epoch 54/150
1051/1051 [==============================] - 0s 148us/step - loss: 0.0281 - mean_squared_error: 0.0281 - val_loss: 0.1109 - val_mean_squared_error: 0.1109
Epoch 55/1

1051/1051 [==============================] - 0s 101us/step - loss: 0.0146 - mean_squared_error: 0.0146 - val_loss: 0.1180 - val_mean_squared_error: 0.1180
Epoch 99/150
1051/1051 [==============================] - 0s 81us/step - loss: 0.0144 - mean_squared_error: 0.0144 - val_loss: 0.1192 - val_mean_squared_error: 0.1192
Epoch 100/150
1051/1051 [==============================] - 0s 101us/step - loss: 0.0141 - mean_squared_error: 0.0141 - val_loss: 0.1192 - val_mean_squared_error: 0.1192
Epoch 101/150
1051/1051 [==============================] - 0s 82us/step - loss: 0.0139 - mean_squared_error: 0.0139 - val_loss: 0.1191 - val_mean_squared_error: 0.1191
Epoch 102/150
1051/1051 [==============================] - 0s 83us/step - loss: 0.0139 - mean_squared_error: 0.0139 - val_loss: 0.1182 - val_mean_squared_error: 0.1182
Epoch 103/150
1051/1051 [==============================] - 0s 88us/step - loss: 0.0138 - mean_squared_error: 0.0138 - val_loss: 0.1185 - val_mean_squared_error: 0.1185
Epoch

Epoch 147/150
1051/1051 [==============================] - 0s 141us/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.1239 - val_mean_squared_error: 0.1239
Epoch 148/150
1051/1051 [==============================] - 0s 103us/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.1246 - val_mean_squared_error: 0.1246
Epoch 149/150
1051/1051 [==============================] - 0s 104us/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.1242 - val_mean_squared_error: 0.1242
Epoch 150/150
1051/1051 [==============================] - 0s 117us/step - loss: 0.0084 - mean_squared_error: 0.0084 - val_loss: 0.1248 - val_mean_squared_error: 0.1248


Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [18]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 66us/step


[0.008212851889902898, 0.008212851889902898]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 86us/step


[0.12478433655638659, 0.12478433655638659]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [20]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, activation='relu',
                             input_shape=(n_features,),
                             kernel_initializer='lecun_normal'))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 267us/step - loss: 0.4771 - mean_squared_error: 0.4771 - val_loss: 0.1465 - val_mean_squared_error: 0.1465
Epoch 2/150
1051/1051 [==============================] - 0s 107us/step - loss: 0.2098 - mean_squared_error: 0.2098 - val_loss: 0.1230 - val_mean_squared_error: 0.1230
Epoch 3/150
1051/1051 [==============================] - 0s 113us/step - loss: 0.1692 - mean_squared_error: 0.1692 - val_loss: 0.1171 - val_mean_squared_error: 0.1171
Epoch 4/150
1051/1051 [==============================] - 0s 91us/step - loss: 0.1476 - mean_squared_error: 0.1476 - val_loss: 0.1085 - val_mean_squared_error: 0.1085
Epoch 5/150
1051/1051 [==============================] - 0s 88us/step - loss: 0.1332 - mean_squared_error: 0.1332 - val_loss: 0.1058 - val_mean_squared_error: 0.1058
Epoch 6/150
1051/1051 [==============================] - 0s 90us/step - loss: 0.1200 - mean_squared_error: 0.1200 - val_

Epoch 50/150
1051/1051 [==============================] - 0s 77us/step - loss: 0.0266 - mean_squared_error: 0.0266 - val_loss: 0.1162 - val_mean_squared_error: 0.1162
Epoch 51/150
1051/1051 [==============================] - 0s 79us/step - loss: 0.0261 - mean_squared_error: 0.0261 - val_loss: 0.1175 - val_mean_squared_error: 0.1175
Epoch 52/150
1051/1051 [==============================] - 0s 78us/step - loss: 0.0258 - mean_squared_error: 0.0258 - val_loss: 0.1188 - val_mean_squared_error: 0.1188
Epoch 53/150
1051/1051 [==============================] - 0s 87us/step - loss: 0.0252 - mean_squared_error: 0.0252 - val_loss: 0.1192 - val_mean_squared_error: 0.1192
Epoch 54/150
1051/1051 [==============================] - 0s 83us/step - loss: 0.0247 - mean_squared_error: 0.0247 - val_loss: 0.1191 - val_mean_squared_error: 0.1191
Epoch 55/150
1051/1051 [==============================] - 0s 112us/step - loss: 0.0243 - mean_squared_error: 0.0243 - val_loss: 0.1189 - val_mean_squared_error: 0.11

Epoch 99/150
1051/1051 [==============================] - 0s 122us/step - loss: 0.0127 - mean_squared_error: 0.0127 - val_loss: 0.1287 - val_mean_squared_error: 0.1287
Epoch 100/150
1051/1051 [==============================] - 0s 117us/step - loss: 0.0125 - mean_squared_error: 0.0125 - val_loss: 0.1283 - val_mean_squared_error: 0.1283
Epoch 101/150
1051/1051 [==============================] - 0s 101us/step - loss: 0.0125 - mean_squared_error: 0.0125 - val_loss: 0.1288 - val_mean_squared_error: 0.1288
Epoch 102/150
1051/1051 [==============================] - 0s 114us/step - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1272 - val_mean_squared_error: 0.1272
Epoch 103/150
1051/1051 [==============================] - 0s 98us/step - loss: 0.0121 - mean_squared_error: 0.0121 - val_loss: 0.1280 - val_mean_squared_error: 0.1280
Epoch 104/150
1051/1051 [==============================] - 0s 81us/step - loss: 0.0120 - mean_squared_error: 0.0120 - val_loss: 0.1299 - val_mean_squared_err

Epoch 148/150
1051/1051 [==============================] - 0s 101us/step - loss: 0.0074 - mean_squared_error: 0.0074 - val_loss: 0.1354 - val_mean_squared_error: 0.1354
Epoch 149/150
1051/1051 [==============================] - 0s 87us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.1338 - val_mean_squared_error: 0.1338
Epoch 150/150
1051/1051 [==============================] - 0s 80us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.1351 - val_mean_squared_error: 0.1351


Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 74us/step


[0.007111399658254705, 0.007111399658254705]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 92us/step


[0.13509990163005803, 0.13509990163005803]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [24]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop',
                     metrics=['mse'],
                     loss='mse')

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 422us/step - loss: 0.3207 - mean_squared_error: 0.3207 - val_loss: 0.1373 - val_mean_squared_error: 0.1373
Epoch 2/150
1051/1051 [==============================] - 0s 94us/step - loss: 0.1663 - mean_squared_error: 0.1663 - val_loss: 0.1099 - val_mean_squared_error: 0.1099
Epoch 3/150
1051/1051 [==============================] - 0s 92us/step - loss: 0.1235 - mean_squared_error: 0.1235 - val_loss: 0.1210 - val_mean_squared_error: 0.1210
Epoch 4/150
1051/1051 [==============================] - 0s 93us/step - loss: 0.0984 - mean_squared_error: 0.0984 - val_loss: 0.1130 - val_mean_squared_error: 0.1130
Epoch 5/150
1051/1051 [==============================] - 0s 152us/step - loss: 0.0887 - mean_squared_error: 0.0887 - val_loss: 0.0989 - val_mean_squared_error: 0.0989
Epoch 6/150
1051/1051 [==============================] - 0s 86us/step - loss: 0.0669 - mean_squared_error: 0.0669 - val_l

Epoch 50/150
1051/1051 [==============================] - 0s 101us/step - loss: 0.0074 - mean_squared_error: 0.0074 - val_loss: 0.0947 - val_mean_squared_error: 0.0947
Epoch 51/150
1051/1051 [==============================] - 0s 92us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1184 - val_mean_squared_error: 0.1184
Epoch 52/150
1051/1051 [==============================] - 0s 82us/step - loss: 0.0102 - mean_squared_error: 0.0102 - val_loss: 0.0847 - val_mean_squared_error: 0.0847
Epoch 53/150
1051/1051 [==============================] - 0s 112us/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.0821 - val_mean_squared_error: 0.0821
Epoch 54/150
1051/1051 [==============================] - 0s 87us/step - loss: 0.0080 - mean_squared_error: 0.0080 - val_loss: 0.0873 - val_mean_squared_error: 0.0873
Epoch 55/150
1051/1051 [==============================] - 0s 80us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.0780 - val_mean_squared_error: 0.0

Epoch 99/150
1051/1051 [==============================] - 0s 116us/step - loss: 0.0065 - mean_squared_error: 0.0065 - val_loss: 0.0770 - val_mean_squared_error: 0.0770
Epoch 100/150
1051/1051 [==============================] - 0s 89us/step - loss: 0.0061 - mean_squared_error: 0.0061 - val_loss: 0.0805 - val_mean_squared_error: 0.0805
Epoch 101/150
1051/1051 [==============================] - 0s 88us/step - loss: 0.0044 - mean_squared_error: 0.0044 - val_loss: 0.0843 - val_mean_squared_error: 0.0843
Epoch 102/150
1051/1051 [==============================] - 0s 88us/step - loss: 0.0046 - mean_squared_error: 0.0046 - val_loss: 0.0838 - val_mean_squared_error: 0.0838
Epoch 103/150
1051/1051 [==============================] - 0s 83us/step - loss: 0.0059 - mean_squared_error: 0.0059 - val_loss: 0.0813 - val_mean_squared_error: 0.0813
Epoch 104/150
1051/1051 [==============================] - 0s 89us/step - loss: 0.0067 - mean_squared_error: 0.0067 - val_loss: 0.0801 - val_mean_squared_error:

Epoch 148/150
1051/1051 [==============================] - 0s 73us/step - loss: 0.0041 - mean_squared_error: 0.0041 - val_loss: 0.0810 - val_mean_squared_error: 0.0810
Epoch 149/150
1051/1051 [==============================] - 0s 101us/step - loss: 0.0044 - mean_squared_error: 0.0044 - val_loss: 0.0813 - val_mean_squared_error: 0.0813
Epoch 150/150
1051/1051 [==============================] - 0s 73us/step - loss: 0.0036 - mean_squared_error: 0.0036 - val_loss: 0.0832 - val_mean_squared_error: 0.0832


Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [26]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 55us/step


[0.005069212051822701, 0.005069212051822701]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [28]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 86us/step


[0.08321716429616109, 0.08321716429616109]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [30]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam',
                   loss='mse',
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 1s 492us/step - loss: 0.3805 - mean_squared_error: 0.3805 - val_loss: 0.1653 - val_mean_squared_error: 0.1653
Epoch 2/150
1051/1051 [==============================] - 0s 139us/step - loss: 0.1610 - mean_squared_error: 0.1610 - val_loss: 0.1333 - val_mean_squared_error: 0.1333
Epoch 3/150
1051/1051 [==============================] - 0s 150us/step - loss: 0.1130 - mean_squared_error: 0.1130 - val_loss: 0.1041 - val_mean_squared_error: 0.1041
Epoch 4/150
1051/1051 [==============================] - 0s 118us/step - loss: 0.0860 - mean_squared_error: 0.0860 - val_loss: 0.1005 - val_mean_squared_error: 0.1005
Epoch 5/150
1051/1051 [==============================] - 0s 119us/step - loss: 0.0740 - mean_squared_error: 0.0740 - val_loss: 0.1011 - val_mean_squared_error: 0.1011
Epoch 6/150
1051/1051 [==============================] - 0s 126us/step - loss: 0.0569 - mean_squared_error: 0.0569 - v

Epoch 50/150
1051/1051 [==============================] - 0s 115us/step - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0980 - val_mean_squared_error: 0.0980
Epoch 51/150
1051/1051 [==============================] - 0s 100us/step - loss: 9.6705e-04 - mean_squared_error: 9.6705e-04 - val_loss: 0.0985 - val_mean_squared_error: 0.0985
Epoch 52/150
1051/1051 [==============================] - 0s 95us/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.0966 - val_mean_squared_error: 0.0966
Epoch 53/150
1051/1051 [==============================] - 0s 87us/step - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.1037 - val_mean_squared_error: 0.1037
Epoch 54/150
1051/1051 [==============================] - 0s 117us/step - loss: 0.0024 - mean_squared_error: 0.0024 - val_loss: 0.0934 - val_mean_squared_error: 0.0934
Epoch 55/150
1051/1051 [==============================] - 0s 82us/step - loss: 0.0044 - mean_squared_error: 0.0044 - val_loss: 0.0991 - val_mean_squared_e

Epoch 99/150
1051/1051 [==============================] - 0s 91us/step - loss: 7.8481e-04 - mean_squared_error: 7.8481e-04 - val_loss: 0.0884 - val_mean_squared_error: 0.0884
Epoch 100/150
1051/1051 [==============================] - 0s 88us/step - loss: 4.9323e-04 - mean_squared_error: 4.9323e-04 - val_loss: 0.0903 - val_mean_squared_error: 0.0903
Epoch 101/150
1051/1051 [==============================] - 0s 84us/step - loss: 3.4135e-04 - mean_squared_error: 3.4135e-04 - val_loss: 0.0877 - val_mean_squared_error: 0.0877
Epoch 102/150
1051/1051 [==============================] - 0s 93us/step - loss: 2.6889e-04 - mean_squared_error: 2.6889e-04 - val_loss: 0.0895 - val_mean_squared_error: 0.0895
Epoch 103/150
1051/1051 [==============================] - 0s 123us/step - loss: 3.5067e-04 - mean_squared_error: 3.5067e-04 - val_loss: 0.0891 - val_mean_squared_error: 0.0891
Epoch 104/150
1051/1051 [==============================] - 0s 89us/step - loss: 3.4035e-04 - mean_squared_error: 3.4035e

1051/1051 [==============================] - 0s 84us/step - loss: 0.0040 - mean_squared_error: 0.0040 - val_loss: 0.0861 - val_mean_squared_error: 0.0861
Epoch 148/150
1051/1051 [==============================] - 0s 86us/step - loss: 0.0036 - mean_squared_error: 0.0036 - val_loss: 0.0849 - val_mean_squared_error: 0.0849
Epoch 149/150
1051/1051 [==============================] - 0s 141us/step - loss: 0.0024 - mean_squared_error: 0.0024 - val_loss: 0.0851 - val_mean_squared_error: 0.0851
Epoch 150/150
1051/1051 [==============================] - 0s 170us/step - loss: 0.0019 - mean_squared_error: 0.0019 - val_loss: 0.0840 - val_mean_squared_error: 0.0840


Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [31]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 86us/step


[0.0015474067065270208, 0.0015474067065270208]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [33]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 78us/step


[0.08395574532959171, 0.08395574532959171]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [34]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

146/146 [==============================] - 0s 123us/step


[0.18289046418176938, 0.18289046418176938]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [35]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

33606.79072320743

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.